In [27]:
import requests
from bs4 import BeautifulSoup as BF
import csv

In [28]:
url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36',
    'Cookie': 'language=en_US',
    'Accept-Language': 'en-US,en;q=0.5'
}

html = requests.get(url, headers=headers)

sp = BF(html.text,'html.parser')

Part A

In [32]:
movie_item = sp.find_all(class_='ipc-metadata-list-summary-item__c')

In [36]:
all_durations = []
with open('../result/films250.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['ranking', 'name', 'year', 'duration', 'rating', 'rate']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
    writer.writeheader()
    
    for movie in movie_item:
        # 處理rank跟name
        name = movie.find(class_='ipc-title__text').text.strip()
        parts = name.split('. ', 1)

        if len(parts) == 2:
            movie_rank = parts[0]
            movie_name = parts[1]  # 獲取電影名部分
        
        # 處理year, duration, rating
        metadata_element = movie.find(class_='sc-6fa21551-7 jLjTzn cli-title-metadata')
        #print(metadata_items)
        metadata_items = metadata_element.find_all(class_='sc-6fa21551-8 bnyjtW cli-title-metadata-item')
        
        year = None
        duration = None
        rating = None
    
        for item in metadata_items:
            text = item.text.strip()
            if 'h' in text or 'm' in text: # 如果有h或m就代表是duration
                duration = text
            elif text.isdigit(): # 如果只有數字就是年份
                year = text
            else:
                rating = text

        # 處理missing data
        movie_year = year if year else "missing data"
        movie_duration = duration if duration else "missing data"
        movie_rating = rating if rating else "missing data"
        # 把時長加進去陣列，part B計算用
        all_durations.append(movie_duration)
        
        # 處理rate
        rate_element = movie.find(class_='ipc-rating-star--base')
        rate = rate_element.get_text(strip=True)
        rate_split = rate.split('(')

        movie_rate = rate_split[0]        
        writer.writerow({'ranking': movie_rank, 'name': movie_name, 'year': movie_year, 
                         'duration': movie_duration, 'rating' : movie_rating, 'rate' : movie_rate})

Part B

In [39]:
total_hours = 0
total_minutes = 0

for durations in all_durations:
    parts = durations.split()
    for part in parts:
        if part.endswith("h"):
            hours = int(part[:-1])  # 取得小時
            total_hours += hours
        elif part.endswith("m"):
            minutes = int(part[:-1])  # 取得分鐘
            total_minutes += minutes

# 把分鐘換成小時，有超過進位
if total_minutes >= 60:
    additional_hours = total_minutes // 60
    total_hours += additional_hours
    total_minutes %= 60


count_duration = f"{total_hours} hours {total_minutes} minutes"
print(count_duration)

537 hours 59 minutes
